# Copying over all forgotten files on the corr nodes

In [1]:
import time
import numpy as np
from subprocess import check_output
from dsautils.dsa_store import DsaStore
from dsautils import cnf
ETCD = DsaStore()

In [2]:
myconf = cnf.Conf()
params = myconf.get('corr')
corrnames = list(params['ch0'].keys())
filenames = [None]*len(corrnames)

In [3]:
for i, corr in enumerate(corrnames):
    try:
        filelist = check_output([
            "rsync",
            "--list-only",
            "{0}.sas.pvt:~/data/*.hdf5".format(corr),
            "--exclude",
            "*incomplete.hdf5"
        ]).decode("utf-8")
        filelist = filelist.split('\n')
        while '' in filelist:
            filelist.remove('')
        filenames[i] = sorted(file.split(' ')[-1] for file in filelist)
    except:
        filenames[i] = []
for fn in filenames:
    fn += ['9999']

In [4]:
sum([len(fn) for fn in filenames])-len(filenames)

187

In [5]:
filenames

[['2021-08-23T07:46:49.hdf5',
  '2021-08-23T07:51:52.hdf5',
  '2021-08-26T09:43:10.hdf5',
  '2021-08-27T07:54:57.hdf5',
  '2021-08-31T08:59:07.hdf5',
  '2021-08-31T09:09:13.hdf5',
  '2021-09-11T14:41:16.hdf5',
  '2021-09-24T17:12:28.hdf5',
  '2021-09-29T12:09:45.hdf5',
  '2021-10-01T14:56:37.hdf5',
  '2021-10-01T15:01:40.hdf5',
  '2021-10-03T19:12:50.hdf5',
  '2021-10-03T19:17:52.hdf5',
  '2021-10-03T19:22:55.hdf5',
  '9999'],
 ['2021-08-23T07:46:47.hdf5',
  '2021-08-23T07:51:50.hdf5',
  '2021-08-26T09:43:09.hdf5',
  '2021-08-27T07:54:56.hdf5',
  '2021-09-11T14:41:15.hdf5',
  '2021-09-24T17:12:28.hdf5',
  '2021-09-29T12:09:43.hdf5',
  '2021-10-03T19:12:48.hdf5',
  '2021-10-03T19:17:51.hdf5',
  '2021-10-03T19:22:53.hdf5',
  '9999'],
 ['2021-08-23T07:46:47.hdf5',
  '2021-08-23T07:51:49.hdf5',
  '2021-08-27T07:54:56.hdf5',
  '2021-09-11T14:41:14.hdf5',
  '2021-09-24T17:12:26.hdf5',
  '2021-09-29T12:09:43.hdf5',
  '2021-10-03T19:12:48.hdf5',
  '2021-10-03T19:17:51.hdf5',
  '2021-10-03T19:2

In [6]:
while sum([len(fn) for fn in filenames]) > len(filenames)*2-1:
    nextfiles = [fn[0] for fn in filenames]
    file = min(nextfiles)
    corridx = nextfiles.index(file)
    if '2021-10' in file:
        ETCD.put_dict(
            '/cmd/cal',
            {'cmd': 'rsync',
             'val': {'hostname': corrnames[corridx],
                     'filename': '/home/ubuntu/data/{0}'.format(file)
            }
            }
        )
    filenames[corridx].remove(file)
    time.sleep(1)

KeyboardInterrupt: 